# Window LAG

**1.The example uses a WHERE clause to show the cases in 'Italy' in March.**  
Modify the query to show data from Spain

In [ ]:
SELECT name, DAY(whn),
 confirmed, deaths, recovered
FROM covid
WHERE name = 'Spain'
AND MONTH(whn) = 3
ORDER BY whn;


**2.The LAG function is used to show data from the preceding row or the table. When lining up rows the data is partitioned by country name and ordered by the data whn. That means that only data from Italy is considered.**  
Modify the query to show confirmed for the day before.

In [ ]:
SELECT name, DAY(whn), confirmed,
   LAG(confirmed,1) OVER (PARTITION BY name ORDER BY whn) PreDay
FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3
ORDER BY whn;

**3.The number of confirmed case is cumulative - but we can use LAG to recover the number of new cases reported for each day.**  
Show the number of new cases for each day, for Italy, for March.

In [ ]:
SELECT name, day, confirmed-lag AS new
FROM(
    SELECT name, DAY(whn) day, confirmed,
    LAG(confirmed, 1) OVER (PARTITION BY name ORDER BY whn) lag  
    FROM covid
    WHERE name = 'Italy'
    AND MONTH(whn) = 3
    ORDER BY whn
    ) temp;

**4.The data gathered are necessarily estimates and are inaccurate. However by taking a longer time span we can mitigate some of the effects.**  
-You can filter the data to view only Monday's figures WHERE WEEKDAY(whn) = 0.  
-Show the number of new cases in Italy for each week - show Monday only.

In [ ]:
SELECT name, whn, (confirmed-lag) AS new
FROM(
    SELECT name, DATE_FORMAT(whn,'%Y-%m-%d') whn, confirmed,
    LAG(confirmed)OVER(PARTITION BY name ORDER BY whn) lag 
    FROM covid
    WHERE name = 'Italy'
    AND WEEKDAY(whn) = 0
    ORDER BY whn
    ) temp;

**5.You can JOIN a table using DATE arithmetic. This will give different results if data is missing.**  
-Show the number of new cases in Italy for each week - show Monday only.  
-In the sample query we JOIN this week tw with last week lw using the DATE_ADD function.

In [ ]:
SELECT tw.name, DATE_FORMAT(tw.whn,'%Y-%m-%d'), 
 tw.confirmed-lw.confirmed
FROM covid tw 
LEFT JOIN covid lw 
ON(DATE_ADD(lw.whn, INTERVAL 1 WEEK) = tw.whn
   AND tw.name=lw.name)
WHERE tw.name = 'Italy' AND WEEKDAY(tw.whn)=0
ORDER BY tw.whn;


**6.The query shown shows the number of confirmed cases together with the world ranking for cases.**  
United States has the highest number, Spain is number 2...  
Notice that while Spain has the second highest confirmed cases, Italy has the second highest number of deaths due to the virus.  
Include the ranking for the number of deaths in the table.

In [ ]:
SELECT 
   name,
   confirmed,
   RANK() OVER (ORDER BY confirmed DESC) rc,
   deaths,
   RANK() OVER (ORDER BY deaths DESC) death_rc
FROM covid
WHERE whn = '2020-04-20'
ORDER BY confirmed DESC;

**7.The query shown includes a JOIN t the world table so we can access the total population of each country and calculate infection rates (in cases per 100,000).**  
Show the infect rate ranking for each country. Only include countries with a population of at least 10 million.

In [ ]:
SELECT world.name,round(100000*confirmed/population,0) AS infect,
       RANK() OVER (ORDER BY confirmed/population) AS rank
FROM covid JOIN world ON covid.name=world.name
WHERE whn = '2020-04-20' AND population > 10000000
ORDER BY population DESC;

**8.For each country that has had at last 1000 new cases in a single day, show the date of the peak number of new cases.**

In [ ]:
SELECT name,date,MAX(confirmed-lag) AS PeakNew 
FROM(
    SELECT name, DATE_FORMAT(whn,'%Y-%m-%d') date, confirmed,
    LAG(confirmed, 1) OVER (PARTITION BY name ORDER BY whn) lag  
    FROM covid 
    ORDER BY  confirmed
) temp
GROUP BY name
HAVING PeakNew>=1000
ORDER BY PeakNew DESC;

# Self Join

**5.Execute the self join shown and observe that b.stop gives all the places you can get to from Craiglockhart, without changing routes. Change the query so that it shows the services from Craiglockhart to London Road.**

In [ ]:
SELECT a.company, a.num, a.stop, b.stop
FROM route a JOIN route b ON
  (a.company=b.company AND a.num=b.num)
WHERE a.stop=53 AND b.stop=149;

**6.The query shown is similar to the previous one, however by joining two copies of the stops table we can refer to stops by name rather than by number. Change the query so that the services between 'Craiglockhart' and 'London Road' are shown.**

In [ ]:
SELECT a.company, a.num, stopa.name, stopb.name
FROM route a JOIN route b ON
  (a.company=b.company AND a.num=b.num)
  JOIN stops stopa ON (a.stop=stopa.id)
  JOIN stops stopb ON (b.stop=stopb.id)
WHERE stopa.name='Craiglockhart'
AND stopb.name='London Road';

**7.Give a list of all the services which connect stops 115 and 137 ('Haymarket' and 'Leith')**

In [ ]:
SELECT DISTINCT(a.company),a.num
FROM route a JOIN route b
ON a.num=b.num AND a.company=b.company
WHERE a.stop=115 AND b.stop=137;

**8.Give a list of the services which connect the stops 'Craiglockhart' and 'Tollcross'**

In [ ]:
SELECT DISTINCT(a.company),a.num
FROM route a 
JOIN route b ON (a.num=b.num AND a.company=b.company)
JOIN stops stopa ON stopa.id=a.stop
JOIN stops stopb ON stopb.id=b.stop
WHERE stopa.name='Craiglockhart'  AND stopb.name='Tollcross';

**9.Give a distinct list of the stops which may be reached from 'Craiglockhart' by taking one bus, including 'Craiglockhart' itself, offered by the LRT company. Include the company and bus no. of the relevant services.**


In [ ]:
SELECT DISTINCT(stopa.name), a.company, a.num
FROM route a 
JOIN route b ON (a.num=b.num AND a.company=b.company)
JOIN stops stopa ON stopa.id=a.stop
JOIN stops stopb ON stopb.id=b.stop
WHERE stopb.name= 'Craiglockhart' ;

** **10.Find the routes involving two buses that can go from Craiglockhart to Lochend.
Show the bus no. and company for the first bus, the name of the stop for the transfer,
and the bus no. and company for the second bus.**  
Hint  
Self-join twice to find buses that visit Craiglockhart and Lochend, then join those on matching stops.

**Steps to solve this problem: divided the problem into two.**.  
-First subquery will be the table S(Start), which will get all the routes that start from 'Craiglockhart' Second subquery will be the table E(End), which will get all the routes that start from 'Lochend'  
-Now both table S and E will have common routes, and i get all this common routes by joining the subqueries, using the ids of each table. As there are duplicates routes(same: S.num, S.company, stops.name, E.num, E.company) i used DISTINCT.

In [ ]:
SELECT DISTINCT S.num, S.company, stops.name, E.num, E.company
FROM
    (SELECT a.company, a.num, b.stop
     FROM route a JOIN route b ON (a.company=b.company AND a.num=b.num)
     WHERE a.stop=(SELECT id FROM stops WHERE name= 'Craiglockhart')
    )S
JOIN
    (SELECT a.company, a.num, b.stop
     FROM route a JOIN route b ON (a.company=b.company AND a.num=b.num)
     WHERE a.stop=(SELECT id FROM stops WHERE name= 'Lochend')
    )E
ON (S.stop = E.stop)
JOIN stops ON(stops.id = S.stop)